# M4L Demucs Worker (One Notebook)

Run all cells once. Drops-only workflow is default. Advanced users can flip `zero_config` in Drive config.


In [ ]:
# Cell 1: Mount Drive and show GPU
from google.colab import drive
import subprocess

drive.mount('/content/drive', force_remount=True)
print(subprocess.getoutput('nvidia-smi'))


In [ ]:
# Cell 2: Install (cached in Drive), prefer GPU wheels if available
import os, sys, subprocess, pathlib, re
SITE = pathlib.Path('/content/drive/MyDrive/M4L-Demucs/.venv/site-packages')
CACHE = pathlib.Path('/content/drive/MyDrive/M4L-Demucs/pip-cache')
SITE.mkdir(parents=True, exist_ok=True)
CACHE.mkdir(parents=True, exist_ok=True)

if str(SITE) not in sys.path:
    sys.path.insert(0, str(SITE))

def pip_install(pkgs, extra_args=None):
    cmd = [sys.executable, "-m", "pip", "install", "-q", "--no-warn-script-location", "--cache-dir", str(CACHE), "--target", str(SITE)]
    if extra_args:
        cmd += extra_args
    cmd += pkgs
    print("Installing:", " ".join(pkgs))
    res = subprocess.run(cmd, text=True)
    if res.returncode != 0:
        print("pip install failed for:", pkgs)

# Try importing first (uses cached site-packages in Drive)
try:
    import demucs, torch  # noqa
    print("demucs already present:", demucs.__version__)
except Exception:
    # Detect CUDA version from nvidia-smi
    smi = subprocess.getoutput("nvidia-smi")
    m = re.search(r"CUDA Version:\s*([0-9.]+)", smi)
    idx = None
    if m:
        ver = m.group(1)
        if ver.startswith("12.6"):
            idx = "https://download.pytorch.org/whl/cu126"
        elif ver.startswith("12.1"):
            idx = "https://download.pytorch.org/whl/cu121"
        elif ver.startswith("11.8"):
            idx = "https://download.pytorch.org/whl/cu118"
    # Upgrade pip tooling
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "pip", "setuptools", "wheel"], text=True)
    # Install torch first (GPU wheels if index detected)
    if idx:
        pip_install(["torch", "torchvision", "torchaudio"], ["--index-url", idx])
    else:
        pip_install(["torch", "torchvision", "torchaudio"])
    # Install demucs and ipython dep
    pip_install(["demucs"])  # unpinned
    pip_install(["jedi>=0.16"])  # quiet fix for IPython warning

import demucs, torch  # noqa
print('demucs', demucs.__version__)
print('torch', torch.__version__, 'cuda', torch.cuda.is_available())
print('site-packages:', SITE)


In [ ]:
# Cell 3: Config reader (zero_config default true)
import json, pathlib
ROOT = pathlib.Path('/content/drive/MyDrive/M4L-Demucs')
ROOT.mkdir(parents=True, exist_ok=True)
CONFIG = ROOT / 'config.json'
if not CONFIG.exists():
    CONFIG.write_text(json.dumps({"zero_config": True}))
print('Config:', CONFIG.read_text())


In [ ]:
# Cell 4: Watcher loop
import urllib.request, subprocess, sys
watcher_url = 'https://raw.githubusercontent.com/VSTOPIA/Doctor-Sample-Unit-DSU/main/colab_watcher.py'
urllib.request.urlretrieve(watcher_url, ROOT / 'colab_watcher.py')
print('Watcher fetched')
!python /content/drive/MyDrive/M4L-Demucs/colab_watcher.py


In [ ]:
# Optional: one-shot run for a specific file (debug)
# Place a file at /content/drive/MyDrive/M4L-Demucs/jobs/audio/YourSong.wav
# Then uncomment and run below:
# import subprocess, sys, pathlib
# IN_WAV = pathlib.Path('/content/drive/MyDrive/M4L-Demucs/jobs/audio/YourSong.wav')
# OUT = pathlib.Path('/content/drive/MyDrive/M4L-Demucs/out/oneshot')
# OUT.mkdir(parents=True, exist_ok=True)
# cmd = [sys.executable,'-m','demucs.separate','-n','htdemucs_ft','-o',str(OUT),'-j','4','--shifts','4','--two-stems','vocals',str(IN_WAV)]
# print(' '.join(cmd))
# subprocess.run(cmd)
